<a href="https://colab.research.google.com/github/WilliamPoe/CSCI-290/blob/main/notebooks/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import math

In [13]:
def cat_entropy(df, target, attribs):
  entropy = 0
  lowest_entropy = float('inf')
  split_point = 0
  lowest_quant_entropy = float('inf')
  quant_attrib = ''
  overall = len( df )
  ## Remove the [2:] ##
  for attrib in attribs[2:]:
    vals = df[attrib].unique()
    # Quantitative Entropy Calc
    if df[attrib].dtype in ['float64', 'int64']:
      if len( df[attrib].unique() ) > 10:
        vals.sort()
        overall = len( df )
        entropies = []
        for val in vals:
          entropy = 0
          left = df[ df[attrib] <= val ][ [attrib,target] ]
          props = left[ target ].value_counts( normalize = True )
          weight = len( left ) / overall
          for prop in props.array:
            entropy = entropy - weight*prop*math.log2( prop )
          right = df[ df[attrib] > val ][ [attrib,target] ]
          props = right[ target ].value_counts( normalize = True )
          weight = len( right ) / overall
          for prop in props.array:
            if prop > 0:
              entropy = entropy - weight*prop*math.log2( prop )
            entropies.append(entropy)
        index = pd.Series( entropies ).idxmin()
        # Check to see if there is a new lowest Entropy
        if entropies[index] < lowest_quant_entropy:
          # Lowest entropy
          lowest_quant_entropy = entropies[index]
          # Split point with lowest entropy
          split_point = vals[index]
          # Attribute that has the split point
          quant_attrib = attrib

    # Categorical Entropy Calc
    else:
      for val in vals:
        subset_size = len(df[ df[attrib] == val ])
        weight = subset_size / overall
        props = df[ df[attrib] == val ][target].value_counts( normalize=True )
        for p in props.array:
          entropy =  entropy - weight*(p*math.log2(p))
      if lowest_entropy > entropy:
        lowest_entropy = entropy
        lowest_attrib = attrib
  # Compares the lowest categorical Entropy with the lowest quantitative Entropy
  if lowest_entropy < lowest_quant_entropy:
    # If categorical entropy is still lowest
    return lowest_attrib, lowest_entropy
  else:
    # If quantitative entropy is lowest
    lowest_entropy = lowest_quant_entropy
    lowest_attrib = quant_attrib
    return lowest_attrib, split_point, lowest_entropy


In [3]:
# possible check what the values are for that attribute and then call a function that does
# categorical or quantitative if it has more than 10 values
# categorical name and entropy
# quantitative name and splitting point

In [30]:
def cat_gini_impurity(df, target, attribs):
  gini = 0
  lowest_gini = float('inf')
  split_point = 0
  lowest_quant_gini = float('inf')
  quant_attrib = ''
  overall = len( df )
  ## Remove the [2:] ##
  for attrib in attribs[2:]:
    vals = df[attrib].unique()

    # Quantitative Gini Impurity Calc
    if df[attrib].dtype in ['float64', 'int64']:
      if len( df[attrib].unique() ) > 10:
        vals.sort()
        overall = len( df )
        gini_impurities = []
        for val in vals:
          quant_gini = 0
          left = df[ df[attrib] <= val ][ [attrib,target] ]
          props = left[ target ].value_counts( normalize = True )
          weight = len( left ) / overall
          for prop in props.array:
            quant_gini += weight*(1-(prop**2))
          right = df[ df[attrib] > val ][ [attrib,target] ]
          props = right[ target ].value_counts( normalize = True )
          weight = len( right ) / overall
          for prop in props.array:
            if prop > 0:
              ### Change calculation ###
              quant_gini += weight*(1-(prop**2))
            gini_impurities.append(quant_gini)
        index = pd.Series( gini_impurities ).idxmin()
        # Check to see if there is a new lowest Gini Impurity
        if gini_impurities[index] < lowest_quant_gini:
          # Lowest Gini Impurity
          print(vals)
          print(index)
          print(len(gini_impurities))
          print(gini_impurities)
          lowest_quant_gini = gini_impurities[index]
          # Split point with lowest Gini Impurity
          split_point = vals[index]
          # Attribute that has the split point
          quant_attrib = attrib

    # Categorical Gini Impurity Calc
    else:
      for val in vals:
        subset_size = len(df[ df[attrib] == val ])
        weight = subset_size / overall
        props = df[ df[attrib] == val ][target].value_counts( normalize=True )
        for p in props.array:
          gini =  weight*(1-(p**2))
      if lowest_gini > gini:
        lowest_gini = gini
        lowest_attrib = attrib
  # Compares the lowest categorical Gini Impurity with the lowest quantitative Gini Impurity
  if lowest_gini < lowest_quant_gini:
    # If categorical Gini Impurity is still lowest
    return lowest_attrib, lowest_gini
  else:
    # If quantitative Gini Impurity is lowest
    lowest_gini = lowest_quant_gini
    lowest_attrib = quant_attrib
    return lowest_attrib, split_point, lowest_gini

In [15]:
def main ():
  df = pd.read_csv("https://github.com/WilliamPoe/CSCI-290/raw/main/Data/ad_click_dataset.csv")
  attribs = []
  print(df.columns)
  target = str(input("What is your target column?: "))
  for col in df.columns:
    if col != target:
      attribs.append(col)
  measure = input("Would you like to use: \n1. Entropy \n2. Gini impurity \nSelection:")
  if measure == '1':
    print(cat_entropy(df, target, attribs))
  else:
    print(cat_gini_impurity(df, target, attribs))

In [16]:
main()

Index(['id', 'full_name', 'age', 'gender', 'device_type', 'ad_position',
       'browsing_history', 'time_of_day', 'click'],
      dtype='object')
What is your target column?: click
Would you like to use: 
1. Entropy 
2. Gini impurity 
Selection:1
('age', 18.0, 0.21468933295874262)


In [31]:
main()

Index(['id', 'full_name', 'age', 'gender', 'device_type', 'ad_position',
       'browsing_history', 'time_of_day', 'click'],
      dtype='object')
What is your target column?: click
Would you like to use: 
1. Entropy 
2. Gini impurity 
Selection:2
[18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53.
 54. 55. 56. 57. 58. 59. 60. 61. 62. 63. 64. nan]
0
92
[0.30912777708420813, 0.7621065345605732, 0.321482219156484, 0.7642444383129681, 0.33760123548602095, 0.7643472985582488, 0.349468320548027, 0.7641527125246255, 0.357489573766411, 0.7642630662019534, 0.3633645265798433, 0.7644740369784568, 0.3732937483613139, 0.7645001497838523, 0.3811983673477196, 0.7645132208566908, 0.3983802069215515, 0.7643165210923583, 0.40408089314437245, 0.764470004201949, 0.41789667178350104, 0.7643281403149695, 0.4315692889751285, 0.7642065489647496, 0.44347418212870426, 0.7640310680347445, 0.4561010123469128, 0.7637684